In [ ]:
import matplotlib
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from sklearn import svm
from jedi.api.refactoring import inline

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score



df = sns.load_dataset('iris')
print(df.head())

col = ['petal_length', 'petal_width', 'species']
print(df.loc[:, col].head())                            #Selección de columnas

df.species.unique()

col = ['petal_length', 'petal_width']
X = df.loc[:, col]

speciesToNum = {
    'setosa': 0,
    'versicolor':1,
    'virginica': 2
}

df['tmp'] = df['species'].map(speciesToNum)
y = df['tmp']
print(y)

# Modelado SVM Debemos establecer
C = 0.001
clf = svm.SVC(kernel = 'linear', C=C)
#clf = svm.LinearSVC(C=C, loss='hinge')
#clf = svm.SVC(kernel='poly', degree=3, C=C)
#clf = svm.SVC(kernel='rbf', gamma=0.7, C=C)

clf.fit(X, y)
print('----------------Predicción SVM------------------------')
clf.predict([[6, 2]])
print(clf.predict([[6, 2]]))
Xv = X.values.reshape(-1, 1)
h = 0.02

# Para graficar
x_min, x_max = Xv.min(), Xv.max() + 1
y_min, y_max = y.min(), y.max() + 1

xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
z = z.reshape(xx.shape)
fig =  plt.figure(figsize=(8, 6))
ax = plt.contourf(xx, yy, z, cmap='afmhot', alpha=0.3)

plt.scatter(X.values[:, 0], X.values[:, 1], c=y, s=80, alpha=0.9, edgecolors='g')
plt.show()


# -------------------------- Linear SVM Implementación
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

#Scale Features
sc_x = StandardScaler()
X_std_train = sc_x.fit_transform(X_train)

C = 1           #c pequeña lleva a más violaciones del margen, c grande menor margen pero pocas violaciones del margen
clf = svm.SVC(kernel='linear', C=C)
clf.fit(X_std_train, y_train)

#Cross Validation
res = cross_val_score(clf, X_std_train, y_train, cv=10, scoring='accuracy')
print('Average Accuracy: \t {0: .4f}'.format(np.mean(res)))
print('Accuracy SD: \t\t {0:.4f}'.format(np.std(res)))

y_train_pred = cross_val_predict(clf, X_std_train, y_train, cv=3)
confusion_matrix(y_train, y_train_pred)

print('Precision Score: \t {0:.4f}'.format(precision_score(y_train, y_train_pred, average='weighted')))
print('Recall Score: \t\t {0: .4f}'.format(recall_score(y_train, y_train_pred, average='weighted')))
print('F1 Score: \t\t {0: .4f}'.format(f1_score(y_train, y_train_pred, average='weighted')))



#Cross Validation within Test Dataset
y_test_pred = cross_val_predict(clf, sc_x.transform(X_test), y_test, cv=3)
confusion_matrix(y_test, y_test_pred)

print('Precision Score: \t {0:.4f}'.format(precision_score(y_test, y_test_pred, average='weighted')))
print('Recall Score: \t\t {0: .4f}'.format(recall_score(y_test, y_test_pred, average='weighted')))
print('F1 Score: \t\t {0: .4f}'.format(f1_score(y_test, y_test_pred, average='weighted')))


#-------------------------------------------- 3. Polynomial Kernel -------------------------------------------------------------
C = 1
clf = svm.SVC(kernel='poly', degree=3, C=C, gamma='auto')
clf.fit(X, y)

Xv = X.values.reshape(-1, 1)
h = 0.02
x_min, x_max =  Xv.min(), Xv.max() + 1
y_min, y_max = y.min(), y.max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
z = z.reshape(xx.shape)
fig = plt.figure(figsize=(8,6))
ax = plt.contourf(xx, yy, z, cmap='afmhot', alpha=0.3)
plt.scatter(X.values[:, 0], X.values[:, 1], c=y, s=80, alpha=0.9, edgecolors='g')

# Polynomial SVM Implementation

sc_x = StandardScaler()
X_std_train = sc_x.fit_transform(X_train)

C = 1
clf = svm.SVC(kernel='poly', degree=10, C=C, gamma='auto')  #5
clf.fit(X_std_train, y_train)

#Cross validation within Train DataSet

res = cross_val_score(clf, X_std_train, y_train, cv=10, scoring='accuracy')
print('Average Accuracy: \t {0:.4f}'.format(np.mean(res)))
print('Accuracy SD: \t\t {0:.4f}'.format(np.std(res)))

y_train_pred = cross_val_predict(clf, X_std_train, y_train, cv=3)
confusion_matrix(y_train, y_train_pred)

print('Precision Score: \t {0:.4f}'.format(precision_score(y_train, y_train_pred, average='weighted')))
print('Recall Score: \t\t {0: .4f}'.format(recall_score(y_train, y_train_pred, average='weighted')))
print('F1 Score: \t\t {0: .4f}'.format(f1_score(y_train, y_train_pred, average='weighted')))

#Cross Validation within Test Dataset
y_test_pred = cross_val_predict(clf, sc_x.transform(X_test), y_test, cv=3)
confusion_matrix(y_test, y_test_pred)

print('Precision Score: \t {0:.4f}'.format(precision_score(y_test, y_test_pred, average='weighted')))
print('Recall Score: \t\t {0: .4f}'.format(recall_score(y_test, y_test_pred, average='weighted')))
print('F1 Score: \t\t {0: .4f}'.format(f1_score(y_test, y_test_pred, average='weighted')))


